# Tarefa 4: Implantar um modelo para inferência assíncrona

## Tarefa 4.1: configuração do ambiente

Instale os pacotes e as dependências.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from botocore.exceptions import ClientError

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

Salve o modelo do laboratório de treinamento e ajuste no bucket-padrão do Amazon Simple Storage Service (Amazon S3). Configure um modelo usando **create_model** e configure **ModelDataUrl** para referenciar o modelo treinado.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

Carregue os registros assíncronos no bucket padrão do Amazon S3.

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="asynchronous_records.csv", Bucket=bucket, Key=f"{prefix}/asynchronous_records.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
input_location = f"s3://{bucket}/{prefix}/asynchronous_records.csv"

## Tarefa 4.2: Criar um endpoint por meio do modelo retreinado e sintetizado fornecido

O Amazon SageMaker Asynchronous Inference é uma capacidade do SageMaker que enfileira as solicitações de entrada e as processa de maneira assíncrona. Essa opção é ideal para solicitações com tamanho grande de payload (até 1 GB), tempo longo de processamento (até 15 minutos) e requisitos de latência em tempo real. Com o Asynchronous Inference, você pode reduzir os custos fazendo auto scaling da contagem de instâncias para zero quando não há nenhuma solicitação a ser processada. Portanto, você só paga quando o endpoint está processando solicitações.

Há três etapas para criar um endpoint assíncrono usando o SDK Python do SageMaker. Estas etapas são as mesmas usadas para os endpoints em tempo real e sem servidor, mas apresentam configurações diferentes:
1. Criar um modelo do SageMaker no SageMaker.
2. Criar uma configuração de endpoint para um endpoint HTTPS.
3. Criar um endpoint HTTPS.

Você já criou um modelo. Agora você já pode criar uma configuração de endpoint e um endpoint. 

Primeiro, defina o nome da configuração de endpoint e o tipo de instância que deseja usar. Depois, chame a API CreateEndpointConfig.

Para criar uma configuração de endpoint, você precisa definir as seguintes opções:
- **VariantName**: o nome da variante de produção (um ou mais modelos em produção).
- **ModelName**: o nome do modelo que você deseja hospedar. Esse é o nome que você especificou quando criou o modelo.
- **InstanceType**: o tipo de instância de computação.
- **S3OutputPath**: o local para carregar as saídas de resposta quando a solicitação não fornecer um local.
- **MaxConcurrentInvocationsPerInstance**: (opcional) o número máximo de solicitações simultâneas enviadas pelo cliente SageMaker ao contêiner de modelo.

Opcionalmente, você também pode definir um NotificationConfig, selecionando um tópico do Amazon Simple Notification Service (Amazon SNS) que poste notificações quando uma solicitação de inferência é bem-sucedida ou falha. Neste laboratório, você não precisa configurar essa opção.

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-asynchronous-endpoint-{}'.format(create_date)                              
output_location = f"s3://{bucket}/{prefix}/output"

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant.
            "InstanceType": "ml.m5.xlarge", # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
            
        } 
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            # Location to upload response outputs when no location is provided in the request.
            "S3OutputPath": output_location
        },
        "ClientConfig": {
            # (Optional) Specify the max number of inflight invocations per instance
            # If no value is provided, Amazon SageMaker chooses an optimal value for you
            "MaxConcurrentInvocationsPerInstance": 4
        }
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

Em seguida, crie um endpoint. Quando você cria um endpoint assíncrono, o SageMaker inicia instâncias de computação de machine learning (ML) e implanta o modelo de acordo com as especificações na configuração. Consulte [Inferência assíncrona](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html) para saber mais sobre as opções disponíveis com endpoints assíncronos.

Quando o endpoint está em serviço, a função auxiliar imprime o Amazon Resource Name (ARN) do endpoint. A criação do endpoint pode exigir até sete minutos para ser executada.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


No SageMaker Studio, você pode visualizar os detalhes do endpoint na aba **Endpoints**.

A próxima etapa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **asynchronous_inference.ipynb** para a lateral ou selecione (clique com o botão direito) a aba **asynchronous_inference.ipynb** e escolha **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora o endpoint.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar o endpoint, retorne ao notebook selecionando a aba **asynchronous_inference.ipynb**.

1. Selecione o ícone **Página inicial do SageMaker**.
2. Escolha **Deployments** (Implantações).
3. Escolha **Endpoints**.

O SageMaker Studio exibe a aba **Endpoints**.

4. Selecione o endpoint que tem **income-model-asynchronous-** na coluna **Nome**.

Se o endpoint não aparecer, selecione o ícone de atualização até que o endpoint apareça na lista.

O SageMaker Studio exibe a aba **ENDPOINT DETAILS** (DETALHES DO ENDPOINT).

5. Selecione a aba **AWS settings** (Configurações da AWS).

Se você abriu o endpoint antes que a criação fosse concluída, selecione o ícone de atualização até que o **Endpoint status** (Status do endpoint) seja alterado de *Creating* para *InService*.

O **Endpoint type** (Tipo de endpoint) está listado como **Async** (Assíncrono). A seção **Endpoint runtime settings** (Configurações de runtime do endpoint) mostra as configurações que você já escolheu no notebook.

## Tarefa 4.3: Invocar um endpoint para inferência assíncrona com registros de cliente assíncronos

Depois de implantar o modelo usando os serviços de host do SageMaker, você poderá testar o modelo nesse endpoint enviando dados a ele.

Para testar um endpoint assíncrono, inclua o local de entrada do Amazon S3 na chamada de API. Para este laboratório, há um arquivo asynchronous_records.csv no bucket do S3 padrão do SageMaker com 100 registros de clientes que você pode usar para testar o endpoint. Se a ação for bem-sucedida, o serviço retornará uma resposta HTTP 202.

In [ ]:
#send-test-file
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)

print(response)

output_key = response['OutputLocation'].split("/", 3)[3]
print('\nThe output key is: {}'.format(output_key))

Verifique o local de saída para saber se a inferência foi processada. Quando ela for processada, imprima as pontuações de previsão de todos os clientes incluídos na invocação.

In [ ]:
#get-output
def get_output():
    while True:
        try:
            return sagemaker.session.Session().read_s3_file(bucket=bucket, key_prefix=output_key)
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("Waiting for output...")
                time.sleep(2)
                continue
            raise

output = get_output()
print(f"Predictions for the 100 customers: {output}")

## Tarefa 4.4: Excluir o endpoint

A limpeza do endpoint pode ser realizada em três etapas. Primeiro, exclua o endpoint. Depois, exclua a configuração do endpoint. Por fim, se você não precisar mais do modelo, exclua-o.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### Conclusão

Parabéns! Você usou o SageMaker para criar um endpoint assíncrono usando o SDK Python do SageMaker e para invocar o endpoint com sucesso.

A próxima tarefa do laboratório se concentra na transformação em lote.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue na **Tarefa 5: Usar a transformação em lote para obter inferências de um conjunto de dados grande**.